In [1]:
import numpy as np 
import pandas as pd
import torch
import tiktoken 

In [ ]:
x = np.array([[1,2,3]])
print(x.shape)
x

In [ ]:
y = np.array([[1],[2],[3]])
print(y.shape)
y

In [ ]:
x + y

In [ ]:
torch.__version__

In [ ]:
torch.cuda.is_available()

In [ ]:
# create a 0D tensor (scalar) from a Python integer
tensor0d = torch.tensor(1)

# create a 1D tensor (vector) from a Python list
tensor1d = torch.tensor([1, 2, 3])

# create a 2D tensor from a nested Python list
tensor2d = torch.tensor([[1, 2, 3], [4, 5, 6]])

# create a 3D tensor from a nested Python list
tensor3d = torch.tensor([[[1, 2], [3, 4]], [[5, 6], [7, 8]]])

In [ ]:
tensor2d

In [ ]:
tensor2d.T

In [ ]:
import torch.nn.functional as F
from torch.autograd import grad

y = torch.tensor([1.0])
x1 = torch.tensor([1.1])
w1 = torch.tensor([2.2], requires_grad=True)
b = torch.tensor([0.0], requires_grad=True)

z = x1 * w1 + b
a = torch.sigmoid(z)

loss = F.binary_cross_entropy(a, y)

loss.backward()

print(w1.grad)
print(b.grad)

In [ ]:
class NeuralNetwork(torch.nn.Module):
    def __init__(self, num_inputs, num_outputs):
        super().__init__()

        self.layers = torch.nn.Sequential(

            # 1st hidden layer
            torch.nn.Linear(num_inputs, 30),
            torch.nn.ReLU(),

            # 2nd hidden layer
            torch.nn.Linear(30, 20),
            torch.nn.ReLU(),

            # output layer
            torch.nn.Linear(20, num_outputs),
        )

    def forward(self, x):
        logits = self.layers(x)
        return logits

In [ ]:
torch.manual_seed(123)
model = NeuralNetwork(50, 3)
print(model)
num_params = sum(
    p.numel() for p in model.parameters() if p.requires_grad
)
print("Total number of trainable model parameters:", num_params)

In [ ]:
model.layers[0].weight

In [ ]:
print(model.layers[0].weight.shape)

In [ ]:
torch.manual_seed(123)

X = torch.rand((1, 50))
with torch.no_grad():
    out = model(X)
print(out)

In [ ]:
X_train = torch.tensor([
    [-1.2, 3.1],
    [-0.9, 2.9],
    [-0.5, 2.6],
    [2.3, -1.1],
    [2.7, -1.5]
])

y_train = torch.tensor([0, 0, 0, 1, 1])

X_test = torch.tensor([
    [-0.8, 2.8],
    [2.6, -1.6],
])

y_test = torch.tensor([0, 1])

In [ ]:
from torch.utils.data import Dataset


class ToyDataset(Dataset):
    def __init__(self, X, y):
        self.features = X
        self.labels = y

    def __getitem__(self, index):
        one_x = self.features[index]
        one_y = self.labels[index]
        return one_x, one_y

    def __len__(self):
        return self.labels.shape[0]


train_ds = ToyDataset(X_train, y_train)
test_ds = ToyDataset(X_test, y_test)

In [ ]:
from torch.utils.data import DataLoader

torch.manual_seed(123)

train_loader = DataLoader(
    dataset=train_ds,
    batch_size=2,
    shuffle=True,
    num_workers=0,
    drop_last=True
)

test_loader = DataLoader(
    dataset=test_ds,
    batch_size=2,
    shuffle=False,
    num_workers=0,
    drop_last=True
)

for idx, (x, y) in enumerate(train_loader):
    print(f"Batch {idx+1}:", x, y)

In [ ]:
torch.manual_seed(123)
model = NeuralNetwork(num_inputs=2,num_outputs=2)

#device = torch.device('cuda')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # considered pytorch best practice
model.to(device)

optimizer = torch.optim.SGD(model.parameters(),lr=0.5)
epochs = 3

for epoch in range(epochs):

    model.train()

    for batch_idx, (features,labels) in enumerate(train_loader):
        features,labels = features.to(device), labels.to(device)

        logits = model(features)
        loss = F.cross_entropy(logits,labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        print(f"Epoch: {epoch+1:03d}/{epochs:03d}" f" | Batch {batch_idx:03d}/{len(train_loader):03d}" f" | Train/Val Loss: {loss:.2f}")
      
    model.eval()

In [ ]:
# can make predictions
model.eval()
with torch.no_grad():
    outputs = model(X_train) # logits
print(outputs)

torch.set_printoptions(sci_mode=False) 
probs = torch.softmax(outputs,dim=1) # class probas
print(probs)

preds = torch.argmax(probs,dim=1) # convert probas to class preds
print(preds)
 
preds = torch.argmax(outputs,dim=1) # could also get preds direct from logits
print(preds)

In [ ]:
(torch.sum(preds == y_train) / len(y_train))*100 # acc

In [ ]:
for key in model.state_dict(): 
    print(key)

In [ ]:
torch.save(model.state_dict(),"model_test.pth")

In [ ]:
model = NeuralNetwork(2,2)
model.load_state_dict(torch.load("model_test.pth", weights_only=True))

In [ ]:
#model = MyTransformerDecoderModel(...)
#model = model.half().cuda()  # Use float16 to save VRAM

# Check memory usage
#print(torch.cuda.memory_allocated() / 1024 ** 2, "MB allocated")

In [ ]:
x = torch.randn(2,2,3)
y = torch.randn(2,2,3)
y = y.view()
print(x)
print(x.shape)
print("")
print(y)
print(y.shape)

In [ ]:
enc = tiktoken.get_encoding("gpt2")
vocab_size = enc.n_vocab
print(f"Vocab size: {vocab_size}")
es = enc.encode("Hello, Marcus Aurelius.")
print(es)
ds = enc.decode(es)
print(ds)

### Multi-head tensor

In [3]:
# optimal mha
# no loops, one big matrix 
B = 2 # batch size
T = 2 # sequence length
d = 3 # emebedding dim 
hd = 4 # head dim, in practice it will be d // n_heads
nh = 3 # num heads

#X = torch.randint(low=1,high=9,size=(B,T,d))
X = torch.randn((B,T,d))

In [ ]:
ll = torch.nn.Linear(d,3*hd*nh,bias=True)
QKV = ll(X)
print(QKV)
QKV.shape

In [ ]:
qkv = QKV.view(B,T,nh,3,hd) 
print(qkv.shape)

#qkv = QKV.view(B,nh,3,T,hd)
#print(qkv.shape)

In [ ]:
qkv[:,:,0,1,:] # K of head 1

In [ ]:
qkv[:,:,:,1,:] # All Ks

In [ ]:
# Q @ K.T for all heads
attn_matrix = qkv[:,:,:,0,:] @ qkv[:,:,:,1,:].transpose(-2,-1)
# [2, 2, 3, 4] @ [2, 2, 4, 3] = [2, 2, 3, 3]
# [B, T, nh, hd] @ [B, T, hd, nh] = [B, T, nh, nh] --> the attention matrix for each head of each sequence of each batch
print(attn_matrix)
print(attn_matrix.shape)

In [ ]:
qkv[:,:,0,0,:] # Q of head 1 across batches

In [ ]:
qkv[:,:,1,0,:] # Q of head 2 across batches

In [ ]:
QKV[:,:,0*hd:1*hd] # Q of head 1 across batches
#QKV[:,:,1*hd:2*hd] # K of head 1
#QKV[:,:,2*hd:3*hd] # V of head 1
#QKV[:,:,3*hd:4*hd] # Q of head 2

In [ ]:
# Q @ K.T
QKV[:,:,:hd] @ QKV[:,:,hd:2*hd].transpose(-2,-1)
# [B,T,hd] @ [B,hd,T] --> [B,T,T]

### Playing around with view()

simulate self attention calculation

In [64]:
total = B*T*hd*nh*3
#QKV = torch.arange(1.0,total+1).view(B,T,nh*3*hd)
QKV = torch.randn(B,T,nh*3*hd)
QKV # output of linear layer [B,T,KQV*nh*hd]

tensor([[[-0.3062,  0.2642,  0.3102,  0.4786, -1.2811, -0.2103, -2.4662,
          -1.2721, -0.3034,  0.6861, -0.3266,  0.7071,  1.0475,  0.1299,
          -0.9075, -0.7080,  1.3434,  1.1133, -1.2462, -0.4755,  1.1179,
           0.8013,  0.3003,  0.8634, -0.4267, -2.0687, -0.7458, -1.4827,
           0.9015, -0.7634, -1.0350,  0.5715, -0.6100,  0.3836,  0.2797,
           0.1567],
         [ 0.5422, -0.1346, -0.9729,  0.6815,  0.3135, -1.6117,  2.7237,
          -0.1971, -0.9549,  0.8888,  0.3955, -0.6993,  0.8184, -0.8732,
           0.9098,  0.1571,  0.6365,  0.0739,  0.6761, -0.0710,  1.7258,
          -1.8626, -1.0199, -0.5793, -0.0284,  0.8317, -0.6985,  0.4344,
          -0.5708,  0.1838, -0.3823, -0.6147, -0.5443, -0.8104, -1.4201,
           0.1152]],

        [[ 0.0757,  0.0736, -0.7300, -0.4633,  1.4882, -0.5096,  1.0091,
          -0.5052,  0.6855, -0.4818,  0.5789, -0.2515,  0.9766,  2.3151,
          -0.8330,  0.3831,  0.9602,  0.3463, -0.6528,  0.1970,  0.4977,
         

In [65]:
qkv = QKV.view(B,nh,T,3,hd) 
# everything grouped by batch -> head -> sequence -> QKV matrix
qkv

tensor([[[[[-0.3062,  0.2642,  0.3102,  0.4786],
           [-1.2811, -0.2103, -2.4662, -1.2721],
           [-0.3034,  0.6861, -0.3266,  0.7071]],

          [[ 1.0475,  0.1299, -0.9075, -0.7080],
           [ 1.3434,  1.1133, -1.2462, -0.4755],
           [ 1.1179,  0.8013,  0.3003,  0.8634]]],


         [[[-0.4267, -2.0687, -0.7458, -1.4827],
           [ 0.9015, -0.7634, -1.0350,  0.5715],
           [-0.6100,  0.3836,  0.2797,  0.1567]],

          [[ 0.5422, -0.1346, -0.9729,  0.6815],
           [ 0.3135, -1.6117,  2.7237, -0.1971],
           [-0.9549,  0.8888,  0.3955, -0.6993]]],


         [[[ 0.8184, -0.8732,  0.9098,  0.1571],
           [ 0.6365,  0.0739,  0.6761, -0.0710],
           [ 1.7258, -1.8626, -1.0199, -0.5793]],

          [[-0.0284,  0.8317, -0.6985,  0.4344],
           [-0.5708,  0.1838, -0.3823, -0.6147],
           [-0.5443, -0.8104, -1.4201,  0.1152]]]],



        [[[[ 0.0757,  0.0736, -0.7300, -0.4633],
           [ 1.4882, -0.5096,  1.0091, -0.5052],


In [66]:
Q = qkv[:,:,:,0,:] # all Q matrices 
print(Q)
print(Q.shape)

tensor([[[[-0.3062,  0.2642,  0.3102,  0.4786],
          [ 1.0475,  0.1299, -0.9075, -0.7080]],

         [[-0.4267, -2.0687, -0.7458, -1.4827],
          [ 0.5422, -0.1346, -0.9729,  0.6815]],

         [[ 0.8184, -0.8732,  0.9098,  0.1571],
          [-0.0284,  0.8317, -0.6985,  0.4344]]],


        [[[ 0.0757,  0.0736, -0.7300, -0.4633],
          [ 0.9766,  2.3151, -0.8330,  0.3831]],

         [[-0.8830,  0.7813,  0.6567, -0.5447],
          [-0.3422, -0.3264,  0.4268,  0.9416]],

         [[-0.3171,  0.9534,  0.1790, -1.4915],
          [ 0.4982, -0.5435,  2.3647, -1.4747]]]])
torch.Size([2, 3, 2, 4])


In [67]:
K = qkv[:,:,:,1,:] # all K matrices
print(K)
print(K.shape)

tensor([[[[-1.2811, -0.2103, -2.4662, -1.2721],
          [ 1.3434,  1.1133, -1.2462, -0.4755]],

         [[ 0.9015, -0.7634, -1.0350,  0.5715],
          [ 0.3135, -1.6117,  2.7237, -0.1971]],

         [[ 0.6365,  0.0739,  0.6761, -0.0710],
          [-0.5708,  0.1838, -0.3823, -0.6147]]],


        [[[ 1.4882, -0.5096,  1.0091, -0.5052],
          [ 0.9602,  0.3463, -0.6528,  0.1970]],

         [[-0.4041,  1.0168, -0.9345,  1.8308],
          [-0.0763, -0.8357, -1.1926, -0.4637]],

         [[-1.9083,  0.6893, -1.6906, -0.8630],
          [ 0.0449,  0.4397, -0.1326, -0.7851]]]])
torch.Size([2, 3, 2, 4])


In [68]:
K_T = qkv[:,:,:,1,:].transpose(-2,-1) # all K matrices transposed
print(K_T)
print(K_T.shape)

tensor([[[[-1.2811,  1.3434],
          [-0.2103,  1.1133],
          [-2.4662, -1.2462],
          [-1.2721, -0.4755]],

         [[ 0.9015,  0.3135],
          [-0.7634, -1.6117],
          [-1.0350,  2.7237],
          [ 0.5715, -0.1971]],

         [[ 0.6365, -0.5708],
          [ 0.0739,  0.1838],
          [ 0.6761, -0.3823],
          [-0.0710, -0.6147]]],


        [[[ 1.4882,  0.9602],
          [-0.5096,  0.3463],
          [ 1.0091, -0.6528],
          [-0.5052,  0.1970]],

         [[-0.4041, -0.0763],
          [ 1.0168, -0.8357],
          [-0.9345, -1.1926],
          [ 1.8308, -0.4637]],

         [[-1.9083,  0.0449],
          [ 0.6893,  0.4397],
          [-1.6906, -0.1326],
          [-0.8630, -0.7851]]]])
torch.Size([2, 3, 4, 2])


In [69]:
V = qkv[:,:,:,2,:]
print(V)
print(V.shape)

tensor([[[[-0.3034,  0.6861, -0.3266,  0.7071],
          [ 1.1179,  0.8013,  0.3003,  0.8634]],

         [[-0.6100,  0.3836,  0.2797,  0.1567],
          [-0.9549,  0.8888,  0.3955, -0.6993]],

         [[ 1.7258, -1.8626, -1.0199, -0.5793],
          [-0.5443, -0.8104, -1.4201,  0.1152]]],


        [[[ 0.6855, -0.4818,  0.5789, -0.2515],
          [ 0.4977, -1.2107,  0.6248, -0.2825]],

         [[-1.2388, -0.0442,  0.0299, -0.1188],
          [ 1.0769, -0.2042, -0.2424, -1.2177]],

         [[-0.5631, -0.5505, -0.3765, -0.9081],
          [ 1.4550,  0.9858, -1.1073, -0.8797]]]])
torch.Size([2, 3, 2, 4])


In [70]:
AM = Q @ K_T
print(AM)
print(AM.shape)
# goal: [B,nh,T,T]

tensor([[[[-1.0371, -0.7314],
          [ 1.7695,  3.0193]],

         [[ 1.1192,  1.4611],
          [ 1.9880, -2.3972]],

         [[ 1.0602, -1.0719],
          [-0.4597,  0.1691]]],


        [[[-0.4275,  0.4835],
          [-0.7605,  2.3587]],

         [[-0.4597, -1.1161],
          [ 1.1314, -0.6466]],

         [[ 2.2468,  1.5522],
          [-4.0506,  0.6275]]]])
torch.Size([2, 3, 2, 2])


In [ ]:
from math import sqrt 

AM = AM / sqrt(d) # scale
mask = torch.triu(torch.ones_like(AM),diagonal=1).bool()
masked_AM = AM.masked_fill(mask,float('-inf'))
attn_weights = torch.nn.functional.softmax(masked_AM,dim=-1)
attn_weights
#attn_weights @ V

tensor([[[[1.0000, 0.0000],
          [0.3270, 0.6730]],

         [[1.0000, 0.0000],
          [0.9263, 0.0737]],

         [[1.0000, 0.0000],
          [0.4102, 0.5898]]],


        [[[1.0000, 0.0000],
          [0.1417, 0.8583]],

         [[1.0000, 0.0000],
          [0.7362, 0.2638]],

         [[1.0000, 0.0000],
          [0.0629, 0.9371]]]])

In [72]:
#torch.Size([2, 3, 2, 2]) masked AM

In [73]:
#out = AM @ V
#print(out)
#print(out.shape)